In [1]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from time import sleep
from IPython.display import clear_output
from collections import namedtuple, deque

from unityagents import UnityEnvironment

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

In [3]:
from buffer import ReplayBuffer
from model import PolicyNetwork

In [4]:
import warnings
warnings.simplefilter('ignore', UserWarning)

In [5]:
plt.style.use('ggplot')

## Set Configs

In [6]:
is_cuda = torch.cuda.is_available()

if is_cuda: device = torch.device('cuda')
else: device = torch.device('cpu')

In [7]:
BATCH_SIZE = 1024
AGENT_BATCH_SIZE = 32
GAMMA = 0.99
TAU = 0.99

LR = 1e-4
EPSILON = 1e-5
WEIGHT_DECAY = 1e-4

EPS_CLIP = 0.2
GRAD_CLIP = 1
NOISE_REDUCE = 0.999

## Set Environment

In [ ]:
env = UnityEnvironment(file_name='./Crawler.app')

brain_name = env.brain_names[0] # get the brain from unity environment
brain = env.brains[brain_name]

In [ ]:
print('Environment Info')
env_info = env.reset(train_mode=False)[brain_name] # reset environment to a new, random state
state = env_info.vector_observations
action_size = brain.vector_action_space_size

print('Number of agents: {}'.format(len(env_info.agents)))
print('State space: {}'.format(state.shape[1]))
print('Action space: {}'.format(action_size))

## Define [PPO](https://arxiv.org/pdf/1707.06347.pdf) Agent

In [8]:
class PPOAgent():
    """The Agent that will interacts with and learns from the environment."""
    
    def __init__(self, env, seed):
        """Initialize an Agent object."""
        
        self.brain = env.brains[env.brain_names[0]] # get the brain from unity environment
        
        self.env_info = env.reset(train_mode=False)[brain_name]
        
        self.num_agents = len(env_info.agents)
        
        self.state_size = env_info.vector_observations.shape[1]
        self.action_size = self.brain.vector_action_space_size
        
        self.gamma = GAMMA
        self.tau = TAU
        self.lr = LR
        self.epsilon = EPSILON
        self.weight_decay = WEIGHT_DECAY
        
        self.eps_clip = EPS_CLIP
        self.grad_clip = GRAD_CLIP
        self.noice_reduce = NOISE_REDUCE
        
        self.policy = PolicyNetwork(self.state_size, self.action_size, self.action_std, seed).to(device)
        self.policy_optimizer = torch.optim.Adam(self.policy.parameters(), lr=self.lr, eps=self.epsilon, weight_decay=self.weight_decay)
        
        self.trajectory = []
        self.std_scale = 1.
        
        # set buffer
        self.buffer = ReplayBuffer(self.batch_size, self.num_agents, random_seed)
        
    def compute_action(self, action_means, scale=1.):
        
        dist = torch.distributions.Normal(action_means, 
                                          F.hardtanh(self.policy.std, min_val=0.06*scale, max_val=0.6*scale))
        actions = dist.sample()
        
        return actions.detach()
    
    def act(self, states):
        
        states = torch.FloatTensor(states).to(device)
        
        self.policy.eval()
        with torch.no_grad():
            action_means = self.policy.act(states)
            
        self.policy.train()
        
        actions = self.compute_action(action_means)
        actions = actions.cpu().data.numpy()
        
    def evaluate_policy(self, state, action):
        
        action_mean = self.policy.act(state)
        
        dist = torch.distributions.Normal(action_means, 
                                          F.hardtanh(self.policy.std, min_val=0.06*scale, max_val=0.6*scale))
        
        actions = dist.sample()
        action_log_probs = dist.log_prob(actions)
        action_log_probs = torch.sum(action_log_probs, dim=1, keepdim=True)
        
        dist_entropies = dist.entropy().mean()
        state_value = self.policy.evaluate(state)
        
        return action_log_probs, state_values, dist_entropies
        
    def memorize_step(self, trajectory):
        
        self.trajectory.append(trajectory)
        
    def memorize_trajectory(self, states):
        
        pending_value = self.policy(states)[-1]
        self.trajectory.append([states, pending_value, None, None, None, None])
        
        processed_trajectory = [None] * (len(self.trajectory) - 1)
        advantages = torch.Tensor(np.zeros((self.num_agents, 1))).to(device)
        returns = pending_values.detach()

        for i in reversed(range(len(self.trajectory) - 1)):
            
            states, values, actions, log_probs, rewards, dones = self.trajectory[i]
            
            states = torch.Tensor(states).to(device)
            actions = torch.Tensor(actions).to(device)
            rewards = torch.Tensor(rewards).unsqueeze(1).to(device)
            dones = torch.Tensor(dones).unsqueeze(1).to(device)
            
            next_values = self.trajectory[i + 1][1]
            returns = rewards + self.gamma * dones * returns
            TD_error = rewards + self.gamma * dones * next_values.detach() - values.detach()
            advantages = advantages * self.tau * self.gamma * dones + TD_error
            
            processed_trajectory[i] = [states, actions, log_probs, returns, advantages]
            
        self.buffer.add(processed_trajectory)
        self.trajectory = []
        
    def learn(self, states):
        
        if len(self.memory) * self.num_agents > BATCH_SIZE * AGENT_BATCH_SIZE:
            
            for states, actions, old_log_probs, returns, advantages in self.memory.sample():
                
                log_probs, state_values, dist_entropies = self.evaluate_policy(states, actions)
                
                ratios = torch.exp(log_probs - old_log_probs.detach())
                
                surrogate1 = ratios * advantages
                surrogate2 = torch.clamp(ratios, 1-self.eps_clip, 1+self.eps_clip) * advantages
                
                policy_loss = -torch.min(surrogate1, surrogate2).mean(0) - 0.01 * dist_entropies.mean()
                value_loss = 0.5 * (returns - state_values).pow(2).mean()
                
                total_loss = policy_loss + value_loss
                
                self.policy_optimizer.zero_grad()
                total_loss.backward()
                nn.utils.clip_grad_norm_(self.policy.parameters(), self.grad_clip)
                self.policy_optimizer.step()
            
            self.buffer.reset()
            
        self.std_scale = self.std_scale * self.noise_reduce

In [ ]:
agent = PPOAgent(env, seed=90)

## Train The Agent

---